# This notebook parses binary CMG simulation results into numpy arrays.

# File format conversion
CMG sr3 --> CMG rwo (by running CMG Results Report software on rwd files) --> numpy array  
sr3 is the binary CMG simulation result file, whereas rwo is the extracted info in the ascii format  
rwd is a configuration file to tell CMG Results Report software how to export results

# Step 0: Notebook setup

In [18]:
import numpy as np
from pathlib import Path
import sys

# add src to path
repo_root = Path.cwd().parent
sys.path.append(str(repo_root / "src"))

################## User Inputs ############################## 
name_prefix = '251209'
n_cases = 2
property_list = ['SG', 'PRES']
################## End of User Inputs #######################

# set up folder paths
base_path = Path('.')
sim_results_folder = base_path/f'{name_prefix}_sherlock'
save_folder = sim_results_folder/f'{name_prefix}_npy'
save_folder.mkdir(parents=True, exist_ok=True)

# Step 1: Convert CMG sr3 files to rwo files

Note this part needs to be run on a machine where CMG Results Report software is installed.

In [19]:
from CMG2npy import generate_CMG_rwd, run_CMG_rwd_report
from tqdm import tqdm

for property in property_list:
    for case_num in tqdm(range(1,n_cases+1),desc=f'Converting sr3 to rwo for keyword {property}'):
        generate_CMG_rwd(
            sr3_folder_path = sim_results_folder,
            case_name = f'case{case_num}',
            property = property,
            is_gmc_property = False,
            precision = 4
        )

        run_CMG_rwd_report(
            rwd_folder_path = sim_results_folder,
            case_name = f'case{case_num}',
            cmg_version = 'ese-ts2win-v2024.20',
        )

        # remove rwd files
        Path(sim_results_folder, f'case{case_num}.rwd').unlink()

print("\nFinished generating rwo files for all cases.")

Converting sr3 to rwo for keyword PRES: 100%|██████████| 2/2 [00:46<00:00, 23.10s/it]


Finished generating rwo files for all cases.


# Step 2: Extract simulation results from rwo files into numpy arrays

Note for the JD_Sula_2025_flow grid (107x117x79), the reservoir layers are k=41-79. The other layers were not simulated by setting ntg=0.  
Results are saved in numpy array as (i, j, k, timestep) for each property.

In [20]:
import numpy as np
from CMG2npy import CMG_rwo2npy
from tqdm import tqdm

for property in property_list:
    for case_num in tqdm(range(1,n_cases+1),desc=f'Generating numpy arrays for keyword {property}'):
        # extract results on all grid cells
        sim_results = CMG_rwo2npy(
            rwo_folder_path = sim_results_folder/'rwo',
            case_name = f'case{case_num}',
            property = property,
            is_save = False,
            save_folder_path = save_folder,
            show_info = False
        )

        # save
        np.save(save_folder/f'case{case_num}_{property}.npy',sim_results)

print("\nFinished generating numpy arrays for all cases.")

Generating numpy arrays for keyword PRES: 100%|██████████| 2/2 [00:32<00:00, 16.45s/it]


Finished generating numpy arrays for all cases.


In [23]:
arr = np.load('251209_sherlock/251209_npy/case1_PRES.npy')
arr.shape

(107, 117, 79, 6)